In [1]:
import xarray as xr
import pandas as pd

In [2]:
from bs4 import BeautifulSoup
import requests

In [3]:
url = 'https://ncei.axiomdatascience.com/atn/test/'

html = requests.get(url).text
soup = BeautifulSoup(html, 'html.parser')

df_out = pd.DataFrame()

files = []

for file in soup.find_all("a"):
    if '.nc' in file['href']:
        file_url = url+file['href']
        files.append(file_url)#print(file_url)

files

['https://ncei.axiomdatascience.com/atn/test/atn_176798_hawksbill-sea-turtle_trajectory_20220801-20221128.nc',
 'https://ncei.axiomdatascience.com/atn/test/atn_178534_hawksbill-sea-turtle_trajectory_20200918-20201124.nc',
 'https://ncei.axiomdatascience.com/atn/test/atn_178544_loggerhead-turtle_trajectory_20210324-20210805.nc',
 'https://ncei.axiomdatascience.com/atn/test/atn_178551_loggerhead-turtle_trajectory_20210327-20210808.nc',
 'https://ncei.axiomdatascience.com/atn/test/atn_181790_green-sea-turtle_trajectory_20191014-20200303.nc',
 'https://ncei.axiomdatascience.com/atn/test/atn_194944_hawksbill-sea-turtle_trajectory_20220728-20230106.nc',
 'https://ncei.axiomdatascience.com/atn/test/atn_34079_caspian-seal_trajectory_20230129-20230228.nc',
 'https://ncei.axiomdatascience.com/atn/test/atn_34084_caspian-seal_trajectory_20230131-20230303.nc']

## Download files

In [5]:
import os

for file in files:
    
    fname = file.split('/')[-1]
    
    if os.path.exists(fname):
        print(f'{fname} exists.')
    else:
        print(f'downloading {fname}')
        r = requests.get(file)
        with open(fname,'wb') as f:
            f.write(r.content)

downloading atn_176798_hawksbill-sea-turtle_trajectory_20220801-20221128.nc
downloading atn_178534_hawksbill-sea-turtle_trajectory_20200918-20201124.nc
downloading atn_178544_loggerhead-turtle_trajectory_20210324-20210805.nc
downloading atn_178551_loggerhead-turtle_trajectory_20210327-20210808.nc
downloading atn_181790_green-sea-turtle_trajectory_20191014-20200303.nc
downloading atn_194944_hawksbill-sea-turtle_trajectory_20220728-20230106.nc
downloading atn_34079_caspian-seal_trajectory_20230129-20230228.nc
downloading atn_34084_caspian-seal_trajectory_20230131-20230303.nc


In [6]:
ds = xr.open_dataset(fname)

ds.info()

xarray.Dataset {
dimensions:
	obs = 115 ;

variables:
	float64 deploy_id() ;
		deploy_id:long_name = id for this deployment. This is typically the tag ptt ;
		deploy_id:comment = Friendly name given to the tag by the user. If no specific friendly name is given, this is the PTT id. ;
		deploy_id:coverage_content_type = referenceInformation ;
	datetime64[ns] time(obs) ;
		time:standard_name = time ;
		time:axis = T ;
		time:_CoordinateAxisType = Time ;
		time:ioos_category = Time ;
		time:long_name = Time of the measurement, in seconds since 1970-01-01 ;
		time:actual_min = 2023-01-31T08:00:00Z ;
		time:actual_max = 2023-03-03T17:21:56Z ;
		time:ancillary_variables = qartod_time_flag qartod_rollup_flag qartod_speed_flag ;
	float64 z(obs) ;
		z:axis = Z ;
		z:long_name = depth of measurement ;
		z:positive = down ;
		z:standard_name = depth ;
		z:units = m ;
		z:actual_min = 0.0 ;
		z:actual_max = 0.0 ;
		z:instrument =  ;
		z:platform = animal ;
		z:comment = This variable is synthetical

In [7]:

for var in ds.variables:
    print(f'\n#### {var}\n')
    print('| attribute | disposition | description | example')
    print('|-----------|-------------|-------------|--------')
    for key in ds[var].attrs:
        value = ds[var].attrs[key]
        print(f'| `{key}` | | | {value}')
    print('\n')
    print('```\n',ds[var],'\n```')


#### deploy_id

| attribute | disposition | description | example
|-----------|-------------|-------------|--------
| `long_name` | | | id for this deployment. This is typically the tag ptt
| `comment` | | | Friendly name given to the tag by the user. If no specific friendly name is given, this is the PTT id.
| `coverage_content_type` | | | referenceInformation


```
 <xarray.DataArray 'deploy_id' ()>
[1 values with dtype=float64]
Attributes:
    long_name:              id for this deployment. This is typically the tag...
    comment:                Friendly name given to the tag by the user. If no...
    coverage_content_type:  referenceInformation 
```

#### time

| attribute | disposition | description | example
|-----------|-------------|-------------|--------
| `standard_name` | | | time
| `axis` | | | T
| `_CoordinateAxisType` | | | Time
| `ioos_category` | | | Time
| `long_name` | | | Time of the measurement, in seconds since 1970-01-01
| `actual_min` | | | 2023-01-31T08:00:00Z

 <xarray.DataArray 'qartod_rollup_flag' (obs: 115)>
[115 values with dtype=float32]
Coordinates:
    time     (obs) datetime64[ns] ...
    z        (obs) float64 ...
    lat      (obs) float64 ...
    lon      (obs) float64 ...
Dimensions without coordinates: obs
Attributes:
    standard_name:   aggregate_quality_flag
    long_name:       Aggregate QC value
    implementation:  https://github.com/ioos/ioos_qc/
    flag_meanings:   PASS NOT_EVALUATED SUSPECT FAIL MISSING
    flag_values:     [1 2 3 4 9]
    references:      https://cdn.ioos.noaa.gov/media/2020/03/QARTOD_TS_Manual... 
```

#### crs

| attribute | disposition | description | example
|-----------|-------------|-------------|--------
| `epsg_code` | | | EPSG:4326
| `grid_mapping_name` | | | latitude_longitude
| `inverse_flattening` | | | 298.257223563
| `ioos_category` | | | Other
| `long_name` | | | Coordinate Reference System - http://www.opengis.net/def/crs/EPSG/0/4326
| `semi_major_axis` | | | 6378137.0
| `coverage_cont

In [40]:
ds['lon'].plot.scatter()

In [31]:
sorted(list(ds.attrs.keys()))

['Conventions',
 'acknowledgement',
 'animal_common_name',
 'animal_id',
 'animal_scientific_name',
 'arbitrary_keywords',
 'argos_program_number',
 'cdm_data_type',
 'citation',
 'comment',
 'contributor_email',
 'contributor_name',
 'contributor_role',
 'contributor_role_vocabulary',
 'creator_country',
 'creator_email',
 'creator_institution',
 'creator_institution_url',
 'creator_name',
 'creator_role',
 'creator_role_vocabulary',
 'creator_sector',
 'creator_sector_vocabulary',
 'creator_type',
 'creator_url',
 'date_created',
 'date_issued',
 'date_metadata_modified',
 'date_modified',
 'deployment_end_datetime',
 'deployment_id',
 'deployment_start_datetime',
 'deployment_start_lat',
 'deployment_start_lon',
 'featureType',
 'geospatial_bbox',
 'geospatial_bounds',
 'geospatial_bounds_crs',
 'geospatial_lat_max',
 'geospatial_lat_min',
 'geospatial_lat_units',
 'geospatial_lon_max',
 'geospatial_lon_min',
 'geospatial_lon_units',
 'history',
 'id',
 'infoUrl',
 'institution',
 '

In [32]:
ds.time.min()

<xarray.DataArray 'time' (obs: 115)>
[115 values with dtype=datetime64[ns]]
Coordinates:
    time     (obs) datetime64[ns] ...
    z        (obs) float64 ...
    lat      (obs) float64 ...
    lon      (obs) float64 ...
Dimensions without coordinates: obs
Attributes:
    standard_name:        time
    axis:                 T
    _CoordinateAxisType:  Time
    ioos_category:        Time
    long_name:            Time of the measurement, in seconds since 1970-01-01
    actual_min:           2023-01-31T08:00:00Z
    actual_max:           2023-03-03T17:21:56Z
    ancillary_variables:  qartod_time_flag qartod_rollup_flag qartod_speed_flag

In [33]:
ds.time.min()

<xarray.DataArray 'time' ()>
array('2023-01-31T08:00:00.000000000', dtype='datetime64[ns]')

In [34]:
ds.time.max()

<xarray.DataArray 'time' ()>
array('2023-03-03T17:21:56.000000000', dtype='datetime64[ns]')

In [39]:
ds['lat'].plot.scatter()